In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
import os
import time
from PIL import Image

In [8]:
# Data Loading and Splitting
data_dir = '/content/drive/MyDrive/JPEGImages'
classes = os.listdir(data_dir)
class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}

def load_data(data_dir, classes):
    images = []
    labels = []
    valid_extensions = ['.jpg', '.jpeg', '.png']
    for cls in classes:
        cls_folder = os.path.join(data_dir, cls)
        for img_filename in os.listdir(cls_folder):
            if any(img_filename.lower().endswith(ext) for ext in valid_extensions):
                img_path = os.path.join(cls_folder, img_filename)
                images.append(img_path)
                labels.append(class_to_idx[cls])
    return train_test_split(images, labels, test_size=0.2, random_state=42)

train_paths, test_paths, train_labels, test_labels = load_data(data_dir, classes)

In [9]:
# Data Preprocessing
transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert('RGB')),  # Convert image to RGB
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

train_dataset = CustomDataset(train_paths, train_labels, transform=transform)
test_dataset = CustomDataset(test_paths, test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8)


In [10]:
# Model Definition
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(classes))


In [11]:
# Device Configuration and Model Training with Mixed Precision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
epochs = 10
scaler = torch.cuda.amp.GradScaler()

start_time = time.time()  # Start timing

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    step_counter = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * images.size(0)
        step_counter += 1

        if step_counter % 20 == 0:
            print(f'Epoch {epoch+1}, Step {step_counter}, Loss: {loss.item():.4f}')

    scheduler.step()
    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch {epoch+1}/{epochs}, Average Loss: {epoch_loss:.4f}')

elapsed_time = time.time() - start_time
print(f'Training completed in: {elapsed_time//60:.0f}m {elapsed_time%60:.0f}s')

Epoch 1, Step 20, Loss: 2.6143
Epoch 1, Step 40, Loss: 2.0254
Epoch 1, Step 60, Loss: 1.9257
Epoch 1, Step 80, Loss: 1.4059
Epoch 1, Step 100, Loss: 1.9724
Epoch 1, Step 120, Loss: 1.6461
Epoch 1/10, Average Loss: 2.0037
Epoch 2, Step 20, Loss: 1.2849
Epoch 2, Step 40, Loss: 1.0207
Epoch 2, Step 60, Loss: 1.1007
Epoch 2, Step 80, Loss: 1.2801
Epoch 2, Step 100, Loss: 1.1736
Epoch 2, Step 120, Loss: 1.3694
Epoch 2/10, Average Loss: 1.2371
Epoch 3, Step 20, Loss: 0.7791
Epoch 3, Step 40, Loss: 0.9270
Epoch 3, Step 60, Loss: 0.5734
Epoch 3, Step 80, Loss: 1.0129
Epoch 3, Step 100, Loss: 0.9868
Epoch 3, Step 120, Loss: 0.6083
Epoch 3/10, Average Loss: 0.9044
Epoch 4, Step 20, Loss: 0.3775
Epoch 4, Step 40, Loss: 0.4220
Epoch 4, Step 60, Loss: 0.5030
Epoch 4, Step 80, Loss: 0.7262
Epoch 4, Step 100, Loss: 0.9308
Epoch 4, Step 120, Loss: 0.9444
Epoch 4/10, Average Loss: 0.6978
Epoch 5, Step 20, Loss: 0.4384
Epoch 5, Step 40, Loss: 0.7181
Epoch 5, Step 60, Loss: 0.4422
Epoch 5, Step 80, Loss:

In [12]:
# Model Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad(), torch.cuda.amp.autocast():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 81.16%
